In [39]:
import re
import pandas as pd

In [40]:
file_path = '/Users/youri/VSC Data/Data Analytics/Fussballdaten/SerieA_data_15_23.xlsx'
df = pd.read_excel(file_path)

# Anzeigen der ersten paar Zeilen der Tabelle zur Überprüfung
df.head()


,matchday_raw,hometeam_raw,awayteam_raw,final_score_raw,goal_raw,goal_time_raw,extratime_raw,Unnamed: 7,Unnamed: 8
0,"Serie A 2014/2015 - 38. Spieltag - So., 31.05....",Sampdoria Genua,FC Parma,2:2,2:2,NaN,NaN,NaN,NaN
1,"Serie A 2014/2015 - 38. Spieltag - So., 31.05....",Sampdoria Genua,FC Parma,2:2,2:1,NaN,NaN,NaN,NaN
2,"Serie A 2014/2015 - 38. Spieltag - So., 31.05....",Sampdoria Genua,FC Parma,2:2,1:1,NaN,NaN,NaN,NaN
3,"Serie A 2014/2015 - 38. Spieltag - So., 31.05....",Sampdoria Genua,FC Parma,2:2,1:0,NaN,NaN,NaN,NaN
4,"Serie A 2014/2015 - 38. Spieltag - So., 31.05....",Sampdoria Genua,FC Parma,2:2,NaN,88',0.0,NaN,NaN


In [41]:
def clean_data(df):
    """
    Bereinigt den DataFrame, indem Zeilen entfernt werden, in denen alle angegebenen Spalten leer sind.

    :param df: Der zu bereinigende DataFrame.
    :return: Der bereinigte DataFrame.
    """

    columns_to_check = ['final_score_raw', 'goal_raw', 'goal_time_raw', 'extratime_raw']
    # Entfernen von Zeilen, in denen alle spezifizierten Spalten leer sind
    df_cleaned = df.dropna(subset=columns_to_check, how='all')
    return df_cleaned

# Beispiel für die Verwendung der Funktion
df_cleaned = clean_data(df)
df_cleaned.head()

,matchday_raw,hometeam_raw,awayteam_raw,final_score_raw,goal_raw,goal_time_raw,extratime_raw,Unnamed: 7,Unnamed: 8
0,"Serie A 2014/2015 - 38. Spieltag - So., 31.05....",Sampdoria Genua,FC Parma,2:2,2:2,NaN,NaN,NaN,NaN
1,"Serie A 2014/2015 - 38. Spieltag - So., 31.05....",Sampdoria Genua,FC Parma,2:2,2:1,NaN,NaN,NaN,NaN
2,"Serie A 2014/2015 - 38. Spieltag - So., 31.05....",Sampdoria Genua,FC Parma,2:2,1:1,NaN,NaN,NaN,NaN
3,"Serie A 2014/2015 - 38. Spieltag - So., 31.05....",Sampdoria Genua,FC Parma,2:2,1:0,NaN,NaN,NaN,NaN
4,"Serie A 2014/2015 - 38. Spieltag - So., 31.05....",Sampdoria Genua,FC Parma,2:2,NaN,88',0.0,NaN,NaN


In [42]:
# Funktionen zur Extraktion von Spieltag und Saison
def extract_spieltag(text):
    match = re.search(r'(\d+)\. Spieltag', text)
    return match.group(1) if match else None

def extract_saison(text):
    match = re.search(r'\b(\d{4}/\d{4})\b', text)
    return match.group(1) if match else None

# Test der Funktionen mit einem Beispieltext
example_text = "Bundesliga 2022/2023 - 34. Spieltag - Sa., 27...."

test_spieltag = extract_spieltag(example_text)
test_saison = extract_saison(example_text)

test_spieltag, test_saison

('34', '2022/2023')

In [44]:
# Erstellen eines DataFrame
df_example = pd.DataFrame(df_cleaned)

# Anwenden der Funktionen zur Erstellung der neuen Spalten
df_example['League'] = 'Serie A'

df_example['matchday_raw'] = df_example['matchday_raw'].astype(str)
df_example['Spieltag'] = df_example['matchday_raw'].apply(extract_spieltag)
df_example['Saison'] = df_example['matchday_raw'].apply(extract_saison)

# Display the first few rows of the updated DataFrame
df_example.head()


,matchday_raw,hometeam_raw,awayteam_raw,final_score_raw,goal_raw,goal_time_raw,extratime_raw,Unnamed: 7,Unnamed: 8,League,Spieltag,Saison
0,"Serie A 2014/2015 - 38. Spieltag - So., 31.05....",Sampdoria Genua,FC Parma,2:2,2:2,NaN,NaN,NaN,NaN,Serie A,38,2014/2015
1,"Serie A 2014/2015 - 38. Spieltag - So., 31.05....",Sampdoria Genua,FC Parma,2:2,2:1,NaN,NaN,NaN,NaN,Serie A,38,2014/2015
2,"Serie A 2014/2015 - 38. Spieltag - So., 31.05....",Sampdoria Genua,FC Parma,2:2,1:1,NaN,NaN,NaN,NaN,Serie A,38,2014/2015
3,"Serie A 2014/2015 - 38. Spieltag - So., 31.05....",Sampdoria Genua,FC Parma,2:2,1:0,NaN,NaN,NaN,NaN,Serie A,38,2014/2015
4,"Serie A 2014/2015 - 38. Spieltag - So., 31.05....",Sampdoria Genua,FC Parma,2:2,NaN,88',0.0,NaN,NaN,Serie A,38,2014/2015


In [45]:
def merge_goal_times(df):
    """
    Updates the extratime_raw column in the DataFrame to contain 0 for 'NO' or empty values, 
    and 1 for 'YES', ensuring that the values from goaltime and extratime of the same row stay together.

    :param df: DataFrame to be processed.
    :return: Processed DataFrame.
    """
    # Group data by matches
    grouped = df.groupby(['matchday_raw', 'hometeam_raw', 'awayteam_raw'])

    # List to store processed data
    merged_data = []

    # Iterate over each group (match)
    for _, group in grouped:
        # Update extratime_raw values: 'YES' -> 1, 'NO' or NaN -> 0
        group['extratime_raw'] = group['extratime_raw'].apply(lambda x: 1 if x == 'YES' else 0)

        # Add the processed data to the list
        merged_data.append(group)

    # Merge all processed matches back into a DataFrame
    return pd.concat(merged_data)

# Example of using the function
# df_example = pd.DataFrame(...)  # Replace with your DataFrame
df_cleaned = merge_goal_times(df_example)
df_cleaned.head(10)


,matchday_raw,hometeam_raw,awayteam_raw,final_score_raw,goal_raw,goal_time_raw,extratime_raw,Unnamed: 7,Unnamed: 8,League,Spieltag,Saison
2185,"Serie A 2014/2015 - 1. Spieltag - Sa., 30.08.2...",AC Chievo Verona,Juventus Turin,0:1,0:1,NaN,0,NaN,NaN,Serie A,1,2014/2015
2186,"Serie A 2014/2015 - 1. Spieltag - Sa., 30.08.2...",AC Chievo Verona,Juventus Turin,0:1,NaN,6',0,NaN,NaN,Serie A,1,2014/2015
2181,"Serie A 2014/2015 - 1. Spieltag - Sa., 30.08.2...",AS Rom,AC Florenz,2:0,2:0,NaN,0,NaN,NaN,Serie A,1,2014/2015
2182,"Serie A 2014/2015 - 1. Spieltag - Sa., 30.08.2...",AS Rom,AC Florenz,2:0,1:0,NaN,0,NaN,NaN,Serie A,1,2014/2015
2183,"Serie A 2014/2015 - 1. Spieltag - Sa., 30.08.2...",AS Rom,AC Florenz,2:0,NaN,90',0,NaN,NaN,Serie A,1,2014/2015
2184,"Serie A 2014/2015 - 1. Spieltag - Sa., 30.08.2...",AS Rom,AC Florenz,2:0,NaN,28',0,NaN,NaN,Serie A,1,2014/2015
2173,"Serie A 2014/2015 - 1. Spieltag - So., 31.08.2...",AC Mailand,Lazio Rom,3:1,3:1,NaN,0,NaN,NaN,Serie A,1,2014/2015
2174,"Serie A 2014/2015 - 1. Spieltag - So., 31.08.2...",AC Mailand,Lazio Rom,3:1,3:0,NaN,0,NaN,NaN,Serie A,1,2014/2015
2175,"Serie A 2014/2015 - 1. Spieltag - So., 31.08.2...",AC Mailand,Lazio Rom,3:1,2:0,NaN,0,NaN,NaN,Serie A,1,2014/2015
2176,"Serie A 2014/2015 - 1. Spieltag - So., 31.08.2...",AC Mailand,Lazio Rom,3:1,1:0,NaN,0,NaN,NaN,Serie A,1,2014/2015


In [46]:
def update_final_score(df):
    """
    Updates the final_score_raw based on the latest goal in each match.
    Matches without goals receive a final_score_raw of '0:0'.

    :param df: DataFrame to be processed.
    :return: Updated DataFrame with the new final_score_raw.
    """
    # Group data by matches
    grouped = df.groupby(['matchday_raw', 'hometeam_raw', 'awayteam_raw'])

    # List to store processed data
    updated_data = []

    # Iterate over each group (match)
    for _, group in grouped:
        # Extract goals and goal times, dropping NaN values
        goals = group['goal_raw'].dropna().tolist()
        times = group['goal_time_raw'].dropna().tolist()

        # Determine the final score of the match
        if goals and times:
            # The last goal (based on time) determines the final score
            final_score = sorted(zip(times, goals))[-1][1]
        else:
            # No goals scored, so the score is '0:0'
            final_score = '0:0'

        # Update final_score_raw for the current match
        game_data = group.copy()
        game_data['final_score_raw'] = final_score

        # Add the processed data to the list
        updated_data.append(game_data)

    # Merge all processed matches back into a DataFrame
    return pd.concat(updated_data)

# Apply the function to your DataFrame
df_updated = update_final_score(df_cleaned)
df_updated.head()


,matchday_raw,hometeam_raw,awayteam_raw,final_score_raw,goal_raw,goal_time_raw,extratime_raw,Unnamed: 7,Unnamed: 8,League,Spieltag,Saison
2185,"Serie A 2014/2015 - 1. Spieltag - Sa., 30.08.2...",AC Chievo Verona,Juventus Turin,0:1,0:1,NaN,0,NaN,NaN,Serie A,1,2014/2015
2186,"Serie A 2014/2015 - 1. Spieltag - Sa., 30.08.2...",AC Chievo Verona,Juventus Turin,0:1,NaN,6',0,NaN,NaN,Serie A,1,2014/2015
2181,"Serie A 2014/2015 - 1. Spieltag - Sa., 30.08.2...",AS Rom,AC Florenz,2:0,2:0,NaN,0,NaN,NaN,Serie A,1,2014/2015
2182,"Serie A 2014/2015 - 1. Spieltag - Sa., 30.08.2...",AS Rom,AC Florenz,2:0,1:0,NaN,0,NaN,NaN,Serie A,1,2014/2015
2183,"Serie A 2014/2015 - 1. Spieltag - Sa., 30.08.2...",AS Rom,AC Florenz,2:0,NaN,90',0,NaN,NaN,Serie A,1,2014/2015


In [47]:
def remove_empty_goals(df):
    """
    Entfernt alle Einträge, bei denen 'goal_raw' leer ist.

    :param df: Der zu bearbeitende DataFrame.
    :return: Der bearbeitete DataFrame ohne leere 'goal_raw' Einträge.
    """
    # Entfernen von Zeilen, in denen 'goal_raw' leer ist
    df_cleaned = df.dropna(subset=['goal_raw'])
    return df_cleaned

# Anwenden der Funktion auf Ihren DataFrame
df_updated = remove_empty_goals(df_updated)
df_updated.head()


,matchday_raw,hometeam_raw,awayteam_raw,final_score_raw,goal_raw,goal_time_raw,extratime_raw,Unnamed: 7,Unnamed: 8,League,Spieltag,Saison
2185,"Serie A 2014/2015 - 1. Spieltag - Sa., 30.08.2...",AC Chievo Verona,Juventus Turin,0:1,0:1,NaN,0,NaN,NaN,Serie A,1,2014/2015
2181,"Serie A 2014/2015 - 1. Spieltag - Sa., 30.08.2...",AS Rom,AC Florenz,2:0,2:0,NaN,0,NaN,NaN,Serie A,1,2014/2015
2182,"Serie A 2014/2015 - 1. Spieltag - Sa., 30.08.2...",AS Rom,AC Florenz,2:0,1:0,NaN,0,NaN,NaN,Serie A,1,2014/2015
2173,"Serie A 2014/2015 - 1. Spieltag - So., 31.08.2...",AC Mailand,Lazio Rom,1:0,3:1,NaN,0,NaN,NaN,Serie A,1,2014/2015
2174,"Serie A 2014/2015 - 1. Spieltag - So., 31.08.2...",AC Mailand,Lazio Rom,1:0,3:0,NaN,0,NaN,NaN,Serie A,1,2014/2015


In [48]:
def rearrange_columns(df):
    """
    Entfernt die Spalte 'matchday_raw' und ändert die Reihenfolge der Spalten.

    :param df: Der zu bearbeitende DataFrame.
    :return: Der bearbeitete DataFrame mit der neuen Spaltenreihenfolge.
    """
    # Entfernen der Spalte 'matchday_raw'
    df = df.drop(columns=['matchday_raw'])

    # Festlegen der neuen Spaltenreihenfolge
    new_order = ['League', 'Saison', 'Spieltag', 'hometeam_raw', 'awayteam_raw',
                 'final_score_raw', 'goal_raw', 'goal_time_raw', 'extratime_raw']

    # Anordnen der Spalten in der neuen Reihenfolge
    df = df[new_order]
    return df

# Anwenden der Funktion auf Ihren DataFrame
df_ord = rearrange_columns(df_updated)
df_ord.head()

,League,Saison,Spieltag,hometeam_raw,awayteam_raw,final_score_raw,goal_raw,goal_time_raw,extratime_raw
2185,Serie A,2014/2015,1,AC Chievo Verona,Juventus Turin,0:1,0:1,NaN,0
2181,Serie A,2014/2015,1,AS Rom,AC Florenz,2:0,2:0,NaN,0
2182,Serie A,2014/2015,1,AS Rom,AC Florenz,2:0,1:0,NaN,0
2173,Serie A,2014/2015,1,AC Mailand,Lazio Rom,1:0,3:1,NaN,0
2174,Serie A,2014/2015,1,AC Mailand,Lazio Rom,1:0,3:0,NaN,0


In [49]:
def save_to_excel(df, path="/Users/youri/VSC Data/Data Analytics/Serie_A_Bereinigt.xlsx"):
    """
    Speichert den übergebenen DataFrame in einer Excel-Datei im angegebenen Verzeichnis.

    :param df: Der DataFrame, der gespeichert werden soll.
    :param path: Der vollständige Pfad und Name der zu erstellenden Excel-Datei.
    """
    try:
        # Speichern des DataFrame in einer Excel-Datei
        df.to_excel(path, index=False)
        return f"Datei '{path}' wurde erfolgreich gespeichert."
    except Exception as e:
        return f"Ein Fehler ist aufgetreten: {e}"

# Speichern des df_example DataFrame in einer Excel-Datei
save_to_excel(df_ord)

"Datei '/Users/youri/VSC Data/Data Analytics/Serie_A_Bereinigt.xlsx' wurde erfolgreich gespeichert."

In [1]:
# Path to the Excel file
excel_path = '/Users/youri/VSC Data/Data Analytics/Fussballdaten/SerieA/Serie_A_Bereinigt.xlsx'  # Update this to your Excel file path

# Read the Excel file
df = pd.read_excel(excel_path)

# Path where you want to save the CSV file
csv_path = '/Users/youri/VSC Data/Data Analytics/Fussballdaten/SerieA/Serie_A_Bereinigt.csv'  # Update this to your desired CSV file path

# Write the DataFrame to a CSV file
df.to_csv(csv_path, index=False)